<a href="https://colab.research.google.com/github/VKSi/2020_12_GB_RecommendationSystems/blob/main/capstone_features_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

# Модель второго уровня
from lightgbm import LGBMClassifier


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
PATH = 'gdrive/My Drive/AI/GB/RecSys/capstone/'
OBJECTS_PATH = PATH + 'objects/'
DATA_PATH = PATH + 'raw_data/'

In [ ]:
def save_obj(obj, name: str):
    path = OBJECTS_PATH + name + '.pkl'
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
    return path

def load_obj(name: str):
    with open(OBJECTS_PATH + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
data = pd.read_csv(DATA_PATH + 'retail_train.csv')
data_lb = pd.read_csv(DATA_PATH + 'retail_test1.csv')

item_features = pd.read_csv(DATA_PATH + 'product.csv')
user_features = pd.read_csv(DATA_PATH + 'hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


In [ ]:
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]

data_train_lvl_2 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

# Генерируем фитчи

In [ ]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [ ]:
data_lb.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0
3,1602,41665647035,664,827939,1,7.99,334,0.0,1741,96,0.0,0.0
4,1602,41665647035,664,927712,1,0.59,334,-0.4,1741,96,0.0,0.0


In [ ]:
us_tr = data.user_id.unique()
len(us_tr)

2499

In [ ]:
us_lb = data_lb.user_id.unique()
len(us_lb)

1885

In [ ]:
set(us_lb) - set(us_tr)

{2325}

Сколько таких товаров купил юзер

In [ ]:
def week_qnt(data_):
  min_week = data_.week_no.min()
  max_week = data_.week_no.max()
  return max_week - min_week + 1

In [ ]:
def tr_ui_1(data_):
  res = data_[['user_id', 'quantity', 'item_id']].groupby(['user_id', 'item_id'], as_index=False).sum()
  res['quantity'] = res['quantity']/week_qnt(data_)
  res.rename(columns={'quantity':'ui_total_quantity'}, inplace=True)
  
  return res

tr_ui_1(data.copy(deep=True)).head()

,user_id,item_id,ui_total_quantity
0,1,819312,0.010526
1,1,820165,0.073684
2,1,821815,0.021053
3,1,821867,0.010526
4,1,823721,0.010526


Сколько таких товаров купил юзер за последние 3 недели

In [ ]:
def tr_ui_2(data_):
  max_week = data_.week_no.max()
  data_ = data_[data_.week_no > max_week - 3]
  res = data_[['user_id', 'quantity', 'item_id']].groupby(['user_id', 'item_id'], as_index=False).sum()
  res['quantity'] = res['quantity']/week_qnt(data_)
  res.rename(columns={'quantity':'3rw_ui_quantity'}, inplace=True)
  return res

tr_ui_2(data.copy(deep=True)).head()

,user_id,item_id,3rw_ui_quantity
0,1,856942,0.333333
1,1,861272,0.333333
2,1,869465,0.333333
3,1,877373,0.333333
4,1,883616,0.333333


Насколько активный юзер

In [ ]:
def tr_u_1(data_):
  res = data_[['user_id', 'basket_id']].groupby(['user_id'], as_index=False).count()
  res['basket_id'] = res['basket_id']/week_qnt(data_)
  res.rename(columns={'basket_id':'basket_quantity'}, inplace=True)
  return res

tr_u_1(data.copy(deep=True)).head()

,user_id,basket_quantity
0,1,16.726316
1,2,6.894737
2,3,9.357895
3,4,3.168421
4,5,2.305263


In [ ]:
def tr_u_2(data_):
  max_week = data_.week_no.max()
  data_ = data_[data_.week_no > max_week - 3]
  res = data_[['user_id', 'basket_id']].groupby(['user_id'], as_index=False).count()
  res['basket_id'] = res['basket_id']/week_qnt(data_)
  res.rename(columns={'basket_id':'3rw_basket_quantity'}, inplace=True)
  return res

tr_u_2(data.copy(deep=True)).head()

,user_id,3rw_basket_quantity
0,1,16.000000
1,6,14.333333
2,7,30.333333
3,8,38.333333
4,9,3.000000


In [ ]:
def tr_u_3(data_):
  res = data_[['user_id', 'sales_value']].groupby(['user_id'], as_index=False).sum()
  res['sales_value'] = res['sales_value']/week_qnt(data_)
  res.rename(columns={'sales_value':'total_sales_value'}, inplace=True)
  return res

tr_u_3(data.copy(deep=True)).head()

,user_id,total_sales_value
0,1,41.683263
1,2,19.194211
2,3,27.308421
3,4,12.632737
4,5,7.885158


In [ ]:
def tr_u_4(data_):
  max_week = data_.week_no.max()
  data_ = data_[data_.week_no > max_week - 3]
  res = data_[['user_id', 'sales_value']].groupby(['user_id'], as_index=False).sum()
  res['sales_value'] = res['sales_value']/week_qnt(data_)
  res.rename(columns={'sales_value':'3rw_sales_value'}, inplace=True)
  return res

tr_u_4(data.copy(deep=True)).head()

,user_id,3rw_sales_value
0,1,43.926667
1,6,48.240000
2,7,81.993333
3,8,95.083333
4,9,8.773333


Покупал ли этот товар за последние n недель

In [ ]:
def tr_ui_3(data_, nw):
  data_['user_item'] = data_.user_id + data_.item_id*10**10
  max_week = data_.week_no.max()
  users_items_nw = data_.loc[data_.week_no > max_week - nw, 'user_item'].unique()
  data_[f'in_{nw}_weeks'] = 0
  data_.loc[data_.user_item.isin(users_items_nw), f'in_{nw}_weeks'] = 1

  res = data_[['user_id', 'item_id', f'in_{nw}_weeks']]
  res = res.groupby(['user_id', 'item_id'], as_index=False).max()
  return res

tr_ui_3(data.copy(deep=True), nw=1).iloc[15:20, :]

,user_id,item_id,in_1_weeks
15,1,830775,0
16,1,831447,0
17,1,831509,0
18,1,831728,0
19,1,832990,0


В каком магазине чаще всего отоваривается юзер

In [ ]:
def tr_u_5(data_):
  res = data_[['user_id', 'store_id', 'basket_id']].groupby(['user_id', 'store_id'], as_index=False).count()
  a = res[['user_id', 'basket_id']].groupby(['user_id'], as_index=False).max()
  a['in'] = a.user_id + a.basket_id*100000
  ain = a['in'].values
  res['in'] = res.user_id + res.basket_id*100000
  res = res.loc[res['in'].isin(ain), ['user_id', 'store_id']]
  res.rename(columns={'store_id':'frq_us_store_id'}, inplace=True)
  return res

tr_u_5(data.copy(deep=True)).head()

,user_id,frq_us_store_id
0,1,436
3,2,401
9,3,401
10,4,298
16,5,374


In [ ]:
def tr_u_6(data_):
  max_week = data_.week_no.max()
  data_ = data_[data_.week_no > max_week - 3]
  res = data_[['user_id', 'store_id', 'basket_id']].groupby(['user_id', 'store_id'], as_index=False).count()
  a = res[['user_id', 'basket_id']].groupby(['user_id'], as_index=False).max()
  a['in'] = a.user_id + a.basket_id*100000
  ain = a['in'].values
  res['in'] = res.user_id + res.basket_id*100000
  res = res.loc[res['in'].isin(ain), ['user_id', 'store_id']]
  res.rename(columns={'store_id':'3rw_frq_us_store_id'}, inplace=True)
  return res

tr_u_6(data.copy(deep=True)).head()

,user_id,3rw_frq_us_store_id
0,1,436
1,6,421
5,7,359
6,8,321
7,9,324


В каком магазине продано больше всего данного товара

In [ ]:
def tr_i_1(data_):
  res = data_[['item_id', 'store_id', 'quantity']].groupby(['item_id', 'store_id'], as_index=False).sum()
  a = res[['item_id', 'quantity']].groupby(['item_id'], as_index=False).max()
  a['in'] = a.item_id + a.quantity*10**10
  ain = a['in'].values
  res['in'] = res.item_id + res.quantity*10**10
  res = res.loc[res['in'].isin(ain), ['item_id', 'store_id']]
  res = res.groupby(['item_id'], as_index=False).max()
  res.rename(columns={'store_id':'frq_itm_store_id'}, inplace=True)
  return res

tr_i_1(data.copy(deep=True)).head()

,item_id,frq_itm_store_id
0,25671,3191
1,26081,3235
2,26093,33904
3,26190,3297
4,26355,3217


In [ ]:
data.item_id.max()

18024556

In [ ]:
def tr_i_2(data_):
  max_week = data_.week_no.max()
  data_ = data_[data_.week_no > max_week - 3]

  res = data_[['item_id', 'store_id', 'quantity']].groupby(['item_id', 'store_id'], as_index=False).sum()
  a = res[['item_id', 'quantity']].groupby(['item_id'], as_index=False).max()

  a['in'] = a.item_id + a.quantity*10**10
  ain = a['in'].values
  res['in'] = res.item_id + res.quantity*10**10
  res = res.loc[res['in'].isin(ain), ['item_id', 'store_id']]
  res = res.groupby(['item_id'], as_index=False).max()
  res.rename(columns={'store_id':'3rw_frq_itm_store_id'}, inplace=True)
  return res

tr_i_2(data.copy(deep=True)).head()

,item_id,3rw_frq_itm_store_id
0,29512,3217
1,32392,3262
2,32439,3072
3,34873,3262
4,40405,3131


In [ ]:
data.store_id.max()

34280

Количество магазинов, в которых продавался товар

In [ ]:
def tr_i_3(data_):
  res = data_[['item_id', 'store_id']].groupby(['item_id'], as_index=False).nunique()
  res.rename(columns={'store_id':'qnt_itm_store_id'}, inplace=True)
  return res

tr_i_3(data.copy(deep=True)).head()

,item_id,qnt_itm_store_id
0,25671,3
1,26081,1
2,26093,1
3,26190,1
4,26355,1


In [ ]:
def tr_i_4(data_):
  max_week = data_.week_no.max()
  data_ = data_[data_.week_no > max_week - 3]
  res = data_[['item_id', 'store_id']].groupby(['item_id'], as_index=False).nunique()
  res.rename(columns={'store_id':'3rw_qnt_itm_store_id'}, inplace=True)
  return res

tr_i_4(data.copy(deep=True)).head()

,item_id,3rw_qnt_itm_store_id
0,29512,1
1,32392,1
2,32439,1
3,34873,1
4,40405,1


Сколько в среднем продавалось товара в неделю

In [ ]:
def tr_i_5(data_):
  res = data_[['item_id', 'quantity']].groupby(['item_id'], as_index=False).sum()
  res['quantity'] = res['quantity']/week_qnt(data_)
  res.rename(columns={'quantity':'wly_quantity'}, inplace=True)
  return res

tr_i_5(data.copy(deep=True)).head()

,item_id,wly_quantity
0,25671,0.063158
1,26081,0.010526
2,26093,0.010526
3,26190,0.010526
4,26355,0.021053


In [ ]:
def tr_i_6(data_):
  max_week = data_.week_no.max()
  data_ = data_[data_.week_no > max_week - 3]
  res = data_[['item_id', 'quantity']].groupby(['item_id'], as_index=False).sum()
  res['quantity'] = res['quantity']/week_qnt(data_)
  res.rename(columns={'quantity':'3rw_wly_quantity'}, inplace=True)
  return res

tr_i_6(data.copy(deep=True)).head()

,item_id,3rw_wly_quantity
0,29512,0.333333
1,32392,0.333333
2,32439,0.333333
3,34873,0.333333
4,40405,0.333333


In [ ]:
def tr_i_7(data_):
  res = data_[['item_id', 'sales_value']].groupby(['item_id'], as_index=False).sum()
  res['sales_value'] = res['sales_value']/week_qnt(data_)
  res.rename(columns={'sales_value':'itm_total_sales_value'}, inplace=True)
  return res

tr_i_7(data.copy(deep=True)).head()

,item_id,itm_total_sales_value
0,25671,0.220421
1,26081,0.010421
2,26093,0.016737
3,26190,0.016211
4,26355,0.020842


In [ ]:
def tr_i_8(data_):
  max_week = data_.week_no.max()
  data_ = data_[data_.week_no > max_week - 3]
  res = data_[['item_id', 'sales_value']].groupby(['item_id'], as_index=False).sum()
  res['sales_value'] = res['sales_value']/week_qnt(data_)
  res.rename(columns={'sales_value':'3rw_itm_total_sales_value'}, inplace=True)
  return res

tr_i_8(data.copy(deep=True)).head()

,item_id,3rw_itm_total_sales_value
0,29512,0.330000
1,32392,0.463333
2,32439,0.626667
3,34873,0.663333
4,40405,0.750000


Готовим таблицы с фичами

In [ ]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [ ]:
item_features.shape

(92353, 7)

In [ ]:
set(data.item_id.unique()) - set(item_features.item_id.unique())

set()

In [ ]:
len(set(item_features.item_id.unique()) - set(data.item_id.unique()))

3302

Все товары, которые есть в тренировочном датасете, есть и в наборе item_features

In [ ]:
set(data_lb.item_id.unique()) - set(item_features.item_id.unique())

set()

"Заглянув в будущее" то же можно сказать и о тестовом сете

In [ ]:
item_features[item_features.item_id==1049998]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
31963,1049998,348,DRUG GM,National,CANDY - PACKAGED,GUM (PACKAGED),


In [ ]:
def create_itf(d_, hidden_weeks):
  d_ = d_[d_.week_no <= d_.week_no.max() - hidden_weeks]
  its_unique = d_.item_id.unique()

  itf = pd.DataFrame({'item_id': its_unique})

  for i in range(8):
    func = eval(f'tr_i_{i+1}')
    itf = itf.merge(func(d_.copy(deep=True)), on='item_id', how='left')

  itf['itm_price'] = itf['itm_total_sales_value']/itf['wly_quantity']

  itf = itf.merge(item_features.loc[item_features.item_id.isin(its_unique), ['item_id', 'department', 'brand']], on='item_id', how='left')
  
  return itf

In [ ]:
items_feats_train = create_itf(data.copy(deep=True), 3)
items_feats_train.head()

,item_id,frq_itm_store_id,3rw_frq_itm_store_id,qnt_itm_store_id,3rw_qnt_itm_store_id,wly_quantity,3rw_wly_quantity,itm_total_sales_value,3rw_itm_total_sales_value,itm_price,department,brand
0,1004906,356,356.0,112,68.0,38.076087,38.000000,90.689783,102.236667,2.381804,PRODUCE,Private
1,1033142,367,34011.0,112,29.0,25.467391,11.333333,24.121522,10.270000,0.947153,PRODUCE,National
2,1036325,447,NaN,76,NaN,2.347826,NaN,3.091413,NaN,1.316713,PRODUCE,Private
3,1082185,367,406.0,113,105.0,298.097826,302.000000,286.356087,304.816667,0.960611,PRODUCE,National
4,8160430,406,34011.0,64,10.0,2.010870,3.333333,3.330326,6.300000,1.656162,PRODUCE,Private


In [ ]:
items_feats_train[items_feats_train.item_id == 1049998]

,item_id,frq_itm_store_id,3rw_frq_itm_store_id,qnt_itm_store_id,3rw_qnt_itm_store_id,wly_quantity,3rw_wly_quantity,itm_total_sales_value,3rw_itm_total_sales_value,itm_price,department,brand
1514,1049998,436,436.0,20,2.0,0.619565,0.666667,1.098587,1.326667,1.773158,DRUG GM,National


In [ ]:
save_obj(items_feats_train, 'items_feats_train_0')

'gdrive/My Drive/AI/GB/RecSys/capstone/objects/items_feats_train_0.pkl'

In [ ]:
data_lb_u = pd.concat([data, data_lb], ignore_index=True)

In [ ]:
data_lb_u.shape[0] == data.shape[0] + data_lb.shape[0]

True

In [ ]:
items_feats_train_lb = create_itf(data_lb_u.copy(deep=True), hidden_weeks=3)  # Хотя мы для удобства и работаем с объединенной базой, значения тестовой баз (3 недели) "скрыты" при тренировке
items_feats_train_lb.head()

,item_id,frq_itm_store_id,3rw_frq_itm_store_id,qnt_itm_store_id,3rw_qnt_itm_store_id,wly_quantity,3rw_wly_quantity,itm_total_sales_value,3rw_itm_total_sales_value,itm_price,department,brand
0,1004906,356,319.0,112,79.0,38.684211,57.333333,92.268737,140.690000,2.385178,PRODUCE,Private
1,1033142,367,319.0,112,25.0,24.957895,9.333333,23.607474,7.843333,0.945892,PRODUCE,National
2,1036325,447,NaN,76,NaN,2.273684,NaN,2.993789,NaN,1.316713,PRODUCE,Private
3,1082185,367,367.0,113,107.0,298.778947,319.666667,287.273895,315.420000,0.961493,PRODUCE,National
4,8160430,406,432.0,67,17.0,2.200000,8.000000,3.701895,15.096667,1.682679,PRODUCE,Private


In [ ]:
save_obj(items_feats_train_lb, 'items_feats_train_lb_0')

'gdrive/My Drive/AI/GB/RecSys/capstone/objects/items_feats_train_lb_0.pkl'

In [ ]:
user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [ ]:
user_features.shape

(801, 8)

In [ ]:
len(set(data.user_id.unique()) - set(user_features.user_id.unique()))

1698

In [ ]:
len(set(data.user_id.unique()))

2499

У нас очень мало дополнительной информации по юзерам.      
Из 2499 уникальных юзеров отсутствует инфо по 1698 (67%)

In [ ]:
user_features.homeowner_desc.value_counts()

Homeowner          504
Unknown            233
Renter              42
Probable Renter     11
Probable Owner      11
Name: homeowner_desc, dtype: int64

In [ ]:
user_features.kid_category_desc.value_counts()

None/Unknown    558
1               114
3+               69
2                60
Name: kid_category_desc, dtype: int64

In [ ]:
user_features.hh_comp_desc.value_counts()

2 Adults No Kids    255
2 Adults Kids       187
Single Female       144
Single Male          95
Unknown              73
1 Adult Kids         47
Name: hh_comp_desc, dtype: int64

In [ ]:
user_features.marital_status_code.value_counts()

U    344
A    340
B    117
Name: marital_status_code, dtype: int64

In [ ]:
user_features.income_desc.value_counts()

50-74K       192
35-49K       172
75-99K        96
25-34K        77
15-24K        74
Under 15K     61
125-149K      38
100-124K      34
150-174K      30
250K+         11
175-199K      11
200-249K       5
Name: income_desc, dtype: int64

In [ ]:
user_features.age_desc.value_counts()

45-54    288
35-44    194
25-34    142
65+       72
55-64     59
19-24     46
Name: age_desc, dtype: int64

Видно, что даже среди имеющихся 32% данных, часть данных также не валидна.     
Если понадобится, в последующем можно будет попытаться использовать данные о возрасте и о доходе. Но изначально попробуем вообще эту таблицу не брать в расчет.

In [ ]:
def create_usf(data_, hidden_weeks):
  data_ = data_[data_.week_no <= data_.week_no.max() - hidden_weeks]
  us_unique = data_.user_id.unique()

  # usf = user_features[user_features.user_id.isin(us_unique)]
  usf = pd.DataFrame({'user_id': us_unique})
  for i in range(6):
    func = eval(f'tr_u_{i+1}')
    usf = usf.merge(func(data_.copy(deep=True)), on='user_id', how='left')

  usf['avg_basket_sum'] = usf['total_sales_value']/usf['basket_quantity']

  return usf

In [ ]:
users_feats_train = create_usf(data.copy(deep=True), 3)
users_feats_train.head()

,user_id,basket_quantity,3rw_basket_quantity,total_sales_value,3rw_sales_value,frq_us_store_id,3rw_frq_us_store_id,avg_basket_sum
0,2375,6.967391,3.333333,26.654457,25.570000,364,364.0,3.825601
1,1364,8.923913,10.000000,24.274891,36.686667,31742,31742.0,2.720207
2,1130,22.239130,8.333333,60.920978,26.870000,31642,31642.0,2.739360
3,1173,2.260870,4.000000,7.127935,10.366667,412,412.0,3.152740
4,98,12.206522,33.666667,22.695217,79.110000,421,421.0,1.859270


In [ ]:
save_obj(users_feats_train, 'users_feats_train_0')

'gdrive/My Drive/AI/GB/RecSys/capstone/objects/users_feats_train_0.pkl'

In [ ]:
users_feats_train_lb = create_usf(data_lb_u.copy(deep=True), hidden_weeks=3)  # Хотя мы для удобства и работаем с объединенной базой, значения тестовой баз (3 недели) "скрыты" при тренировке
users_feats_train_lb.head()

,user_id,basket_quantity,3rw_basket_quantity,total_sales_value,3rw_sales_value,frq_us_store_id,3rw_frq_us_store_id,avg_basket_sum
0,2375,6.915789,5.333333,26.172842,11.403333,364,364.0,3.784505
1,1364,9.105263,14.666667,24.822632,41.620000,31742,31742.0,2.726185
2,1130,22.326316,25.000000,61.556947,81.060000,31642,31642.0,2.757148
3,1173,2.189474,NaN,6.902842,NaN,412,NaN,3.152740
4,98,12.315789,15.666667,23.025053,33.140000,421,421.0,1.869556


In [ ]:
save_obj(users_feats_train_lb, 'users_feats_train_lb_0')

'gdrive/My Drive/AI/GB/RecSys/capstone/objects/users_feats_train_lb_0.pkl'

# Загружаем данные с первого уровня

In [ ]:
users_lvl_2 = load_obj('users_lvl_2_obj')
users_lvl_2_lb = load_obj('users_lvl_2_lb_obj')

In [ ]:
df=pd.DataFrame({'user_id':users_lvl_2.user_id.values.repeat(len(users_lvl_2.candidates[0])),
                 'item_id':np.concatenate(users_lvl_2.candidates.values)})

In [ ]:
df_lb=pd.DataFrame({'user_id':users_lvl_2_lb.user_id.values.repeat(len(users_lvl_2_lb.candidates[0])),
                 'item_id':np.concatenate(users_lvl_2_lb.candidates.values)})

In [ ]:
df.head()

,user_id,item_id
0,1,1049998
1,1,900875
2,1,1127831
3,1,852662
4,1,1082185


In [ ]:
df.loc[df.user_id==1, 'item_id'].nunique()

100

In [ ]:
df.user_id.nunique()

2280

In [ ]:
df_train = df[df.user_id.isin(data_train_lvl_2.user_id.values)]

In [ ]:
df_train.user_id.nunique()

2154

In [ ]:
data_train_lvl_2.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2104867,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,0.0,0.0
2107468,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,0.0,0.0
2107469,2021,40618753059,594,856060,1,1.77,443,-0.09,101,86,0.0,0.0
2107470,2021,40618753059,594,869344,1,1.67,443,-0.22,101,86,0.0,0.0
2107471,2021,40618753059,594,896862,2,5.00,443,-2.98,101,86,0.0,0.0


In [ ]:
targets_train_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_train_lvl_2['target'] = 1  # тут только покупки
targets_train_lvl_2 = targets_train_lvl_2.groupby(['user_id', 'item_id'], as_index=False).min()  # В материалах вебинаров этот шаг был пропущен, данные были не уникальны!
targets_train_lvl_2 = df_train.merge(targets_train_lvl_2, on=['user_id', 'item_id'], how='left')
targets_train_lvl_2['target'].fillna(0, inplace= True)
targets_train_lvl_2.head()

,user_id,item_id,target
0,1,1049998,1.0
1,1,900875,0.0
2,1,1127831,0.0
3,1,852662,0.0
4,1,1082185,1.0


In [ ]:
targets_train_lvl_2['target'].mean()

0.08435933147632312

# Добавление фитчей

In [ ]:
items_feats_train.head()

,item_id,frq_itm_store_id,3rw_frq_itm_store_id,qnt_itm_store_id,3rw_qnt_itm_store_id,wly_quantity,3rw_wly_quantity,itm_total_sales_value,3rw_itm_total_sales_value,itm_price,department,brand
0,1004906,356,356.0,112,68.0,38.076087,38.000000,90.689783,102.236667,2.381804,PRODUCE,Private
1,1033142,367,34011.0,112,29.0,25.467391,11.333333,24.121522,10.270000,0.947153,PRODUCE,National
2,1036325,447,NaN,76,NaN,2.347826,NaN,3.091413,NaN,1.316713,PRODUCE,Private
3,1082185,367,406.0,113,105.0,298.097826,302.000000,286.356087,304.816667,0.960611,PRODUCE,National
4,8160430,406,34011.0,64,10.0,2.010870,3.333333,3.330326,6.300000,1.656162,PRODUCE,Private


In [ ]:
targets_train_lvl_2 = targets_train_lvl_2.merge(items_feats_train, on='item_id', how='left')

In [ ]:
targets_train_lvl_2.head()

,user_id,item_id,target,frq_itm_store_id,3rw_frq_itm_store_id,qnt_itm_store_id,3rw_qnt_itm_store_id,wly_quantity,3rw_wly_quantity,itm_total_sales_value,3rw_itm_total_sales_value,itm_price,department,brand
0,1,1049998,1.0,436.0,436.0,20.0,2.0,0.619565,0.666667,1.098587,1.326667,1.773158,DRUG GM,National
1,1,900875,0.0,313.0,318.0,16.0,1.0,0.510870,0.666667,1.063152,1.526667,2.081064,GROCERY,National
2,1,1127831,0.0,406.0,31862.0,112.0,32.0,81.565217,23.333333,181.095109,113.100000,2.220249,PRODUCE,National
3,1,852662,0.0,436.0,NaN,9.0,NaN,0.445652,NaN,0.701087,NaN,1.573171,DRUG GM,National
4,1,1082185,1.0,367.0,406.0,113.0,105.0,298.097826,302.000000,286.356087,304.816667,0.960611,PRODUCE,National


In [ ]:
users_feats_train.head()

,user_id,basket_quantity,3rw_basket_quantity,total_sales_value,3rw_sales_value,frq_us_store_id,3rw_frq_us_store_id,avg_basket_sum
0,2375,6.967391,3.333333,26.654457,25.570000,364,364.0,3.825601
1,1364,8.923913,10.000000,24.274891,36.686667,31742,31742.0,2.720207
2,1130,22.239130,8.333333,60.920978,26.870000,31642,31642.0,2.739360
3,1173,2.260870,4.000000,7.127935,10.366667,412,412.0,3.152740
4,98,12.206522,33.666667,22.695217,79.110000,421,421.0,1.859270


In [ ]:
targets_train_lvl_2 = targets_train_lvl_2.merge(users_feats_train, on='user_id', how='left')

In [ ]:
def merge_us_it(data_, hidden_weeks, merge_to_data):
  data_ = data_[data_.week_no <= data_.week_no.max() - hidden_weeks]
  merge_to_data_ = merge_to_data.copy(deep=True)
  merge_to_data_ = merge_to_data_.merge(tr_ui_1(data_), on=['user_id', 'item_id'], how='left')
  merge_to_data_ = merge_to_data_.merge(tr_ui_2(data_), on=['user_id', 'item_id'], how='left')
  merge_to_data_ = merge_to_data_.merge(tr_ui_3(data_, nw=3), on=['user_id', 'item_id'], how='left')
  merge_to_data_ = merge_to_data_.merge(tr_ui_3(data_, nw=1), on=['user_id', 'item_id'], how='left')
  return merge_to_data_

In [ ]:
targets_train_lvl_2 = merge_us_it(data.copy(deep=True), 3, targets_train_lvl_2.copy(deep=True))
targets_train_lvl_2.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,user_id,item_id,target,frq_itm_store_id,3rw_frq_itm_store_id,qnt_itm_store_id,3rw_qnt_itm_store_id,wly_quantity,3rw_wly_quantity,itm_total_sales_value,3rw_itm_total_sales_value,itm_price,department,brand,basket_quantity,3rw_basket_quantity,total_sales_value,3rw_sales_value,frq_us_store_id,3rw_frq_us_store_id,avg_basket_sum,ui_total_quantity,3rw_ui_quantity,in_3_weeks,in_1_weeks
0,1,1049998,1.0,436.0,436.0,20.0,2.0,0.619565,0.666667,1.098587,1.326667,1.773158,DRUG GM,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,0.271739,0.333333,1.0,0.0
1,1,900875,0.0,313.0,318.0,16.0,1.0,0.510870,0.666667,1.063152,1.526667,2.081064,GROCERY,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,0.097826,NaN,0.0,0.0
2,1,1127831,0.0,406.0,31862.0,112.0,32.0,81.565217,23.333333,181.095109,113.100000,2.220249,PRODUCE,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,NaN,NaN,NaN,NaN
3,1,852662,0.0,436.0,NaN,9.0,NaN,0.445652,NaN,0.701087,NaN,1.573171,DRUG GM,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,0.260870,NaN,0.0,0.0
4,1,1082185,1.0,367.0,406.0,113.0,105.0,298.097826,302.000000,286.356087,304.816667,0.960611,PRODUCE,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,0.423913,0.666667,1.0,1.0


# Готовим данные и тренируем второй уровень

In [ ]:
X_train = targets_train_lvl_2.drop(columns=['user_id', 'item_id', 'target'], axis=1)
y_train = targets_train_lvl_2[['target']]

In [ ]:
X_train.columns

Index(['frq_itm_store_id', '3rw_frq_itm_store_id', 'qnt_itm_store_id',
       '3rw_qnt_itm_store_id', 'wly_quantity', '3rw_wly_quantity',
       'itm_total_sales_value', '3rw_itm_total_sales_value', 'itm_price',
       'department', 'brand', 'basket_quantity', '3rw_basket_quantity',
       'total_sales_value', '3rw_sales_value', 'frq_us_store_id',
       '3rw_frq_us_store_id', 'avg_basket_sum', 'ui_total_quantity',
       '3rw_ui_quantity', 'in_3_weeks', 'in_1_weeks'],
      dtype='object')

In [ ]:
cat_feats = ['frq_itm_store_id', '3rw_frq_itm_store_id', 
       'department', 'brand', 'frq_us_store_id',
       '3rw_frq_us_store_id']
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['frq_itm_store_id',
 '3rw_frq_itm_store_id',
 'department',
 'brand',
 'frq_us_store_id',
 '3rw_frq_us_store_id']

In [ ]:
y_train.mean()

target    0.083966
dtype: float64

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats, weght='balanced')
lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)[:, 1]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


In [ ]:
train_preds

array([9.91879571e-01, 2.01443294e-01, 2.90538696e-05, ...,
       2.34671906e-05, 2.39534378e-05, 5.55166806e-02])

In [ ]:
k = 5
targets_lvl_2_pred = targets_train_lvl_2.copy(deep=True)
targets_lvl_2_pred['predict_proba'] = train_preds
targets_lvl_2_pred = targets_lvl_2_pred[['user_id', 'item_id', 'predict_proba']]
users = targets_lvl_2_pred['user_id'].unique()
lvl_2_recommendations = pd.DataFrame(data = {'user_id': users})
lvl_2_recommendations['candidates'] = 0

def get_lvl_2_reccomendations(user_id, targets_lvl_2_pred):
  tdf = targets_lvl_2_pred[targets_lvl_2_pred['user_id']==user_id]
  tdf = tdf.drop(columns=['user_id'])
  tdf = tdf.groupby(['item_id']).max()
  tdf = tdf.sort_values(by='predict_proba', ascending=False)
  tdf = tdf.index[:k].tolist()
  return tdf

print(get_lvl_2_reccomendations(2027, targets_lvl_2_pred))
lvl_2_recommendations['candidates'] = lvl_2_recommendations['user_id'].apply(lambda x: get_lvl_2_reccomendations(x, targets_lvl_2_pred))

[1068719, 961554, 1055646, 923746, 981760]


In [ ]:
lvl_2_recommendations.head()

,user_id,candidates
0,1,"[1049998, 840361, 9655212, 8293439, 5577022]"
1,2,"[1133018, 916122, 1053690, 5592931, 1106523]"
2,4,"[891423, 883932, 831063, 1029743, 13115481]"
3,6,"[1082185, 13131271, 995242, 1037863, 878996]"
4,7,"[1133018, 949836, 995785, 1082185, 1106523]"


In [ ]:
lvl_2_recommendations[lvl_2_recommendations.user_id == 1984]

,user_id,candidates
1715,1984,"[995242, 999971, 1098066, 866227, 1056509]"


Результат на трейне 2-го уровня:

In [ ]:
actual_data_train_lvl_2 = data_train_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
actual_data_train_lvl_2.columns=['user_id', 'actual']
actual_data_train_lvl_2.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [ ]:
def get_bought_list(actual_data, user_id):
  return actual_data.loc[actual_data['user_id'] == user_id, 'actual'].values[0]

def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    bought_list = bought_list  # Тут нет [:k] !!
    if k < len(recommended_list):
        recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    return precision

# Определим функцию для расчета precision_at_k по готовому предсказанию
def total_precision_at_k_pre(actual_data, k, recommendations_table):
  pr = []
  for uid in actual_data['user_id'].values:
    if uid == 1984 or uid == 2325:
      continue
    rl = recommendations_table.loc[recommendations_table.user_id == uid, 'candidates'].values[0]
    bl = get_bought_list(actual_data, user_id=uid)
    pr.append(precision_at_k(recommended_list=rl, bought_list=bl, k=k))
  return np.mean(pr), np.std(pr)

In [ ]:
mean, std = total_precision_at_k_pre(actual_data = actual_data_train_lvl_2, k=5, recommendations_table=lvl_2_recommendations)
print(mean, ' +- ', std)

0.696144914073386  +-  0.3516387240957705


Посмотрим на валидационный сет

In [ ]:
actual_data_val_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
actual_data_val_lvl_2.columns=['user_id', 'actual']
actual_data_val_lvl_2.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [ ]:
df_val = df[df.user_id.isin(data_val_lvl_2.user_id.values)]

Расчитаем рекоммендации второго уровня для валидационного набора данных

In [ ]:
targets_val_lvl_2 = data_val_lvl_2[['user_id', 'item_id']].copy()
targets_val_lvl_2['target'] = 1  # тут только покупки
targets_val_lvl_2 = targets_val_lvl_2.groupby(['user_id', 'item_id'], as_index=False).min()  # В материалах вебинаров этот шаг был пропущен, данные были не уникальны!
targets_val_lvl_2 = df_val.merge(targets_val_lvl_2, on=['user_id', 'item_id'], how='left')
targets_val_lvl_2['target'].fillna(0, inplace= True)
targets_val_lvl_2.head()

targets_val_lvl_2 = targets_val_lvl_2.merge(items_feats_train, on='item_id', how='left')
targets_val_lvl_2 = targets_val_lvl_2.merge(users_feats_train, on='user_id', how='left')
targets_val_lvl_2 = merge_us_it(data.copy(deep=True), 3, targets_val_lvl_2.copy(deep=True))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [ ]:
targets_val_lvl_2.head()

,user_id,item_id,target,frq_itm_store_id,3rw_frq_itm_store_id,qnt_itm_store_id,3rw_qnt_itm_store_id,wly_quantity,3rw_wly_quantity,itm_total_sales_value,3rw_itm_total_sales_value,itm_price,department,brand,basket_quantity,3rw_basket_quantity,total_sales_value,3rw_sales_value,frq_us_store_id,3rw_frq_us_store_id,avg_basket_sum,ui_total_quantity,3rw_ui_quantity,in_3_weeks,in_1_weeks
0,1,1049998,1.0,436.0,436.0,20.0,2.0,0.619565,0.666667,1.098587,1.326667,1.773158,DRUG GM,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,0.271739,0.333333,1.0,0.0
1,1,900875,0.0,313.0,318.0,16.0,1.0,0.510870,0.666667,1.063152,1.526667,2.081064,GROCERY,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,0.097826,NaN,0.0,0.0
2,1,1127831,0.0,406.0,31862.0,112.0,32.0,81.565217,23.333333,181.095109,113.100000,2.220249,PRODUCE,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,NaN,NaN,NaN,NaN
3,1,852662,0.0,436.0,NaN,9.0,NaN,0.445652,NaN,0.701087,NaN,1.573171,DRUG GM,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,0.260870,NaN,0.0,0.0
4,1,1082185,1.0,367.0,406.0,113.0,105.0,298.097826,302.000000,286.356087,304.816667,0.960611,PRODUCE,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,0.423913,0.666667,1.0,1.0


In [ ]:
X_val = targets_val_lvl_2.drop(columns=['user_id', 'item_id', 'target'])
X_val[cat_feats] = X_val[cat_feats].astype('category')

val_preds = lgb.predict_proba(X_val)[:, 1]

In [ ]:
X_val.head()

,frq_itm_store_id,3rw_frq_itm_store_id,qnt_itm_store_id,3rw_qnt_itm_store_id,wly_quantity,3rw_wly_quantity,itm_total_sales_value,3rw_itm_total_sales_value,itm_price,department,brand,basket_quantity,3rw_basket_quantity,total_sales_value,3rw_sales_value,frq_us_store_id,3rw_frq_us_store_id,avg_basket_sum,ui_total_quantity,3rw_ui_quantity,in_3_weeks,in_1_weeks
0,436.0,436.0,20.0,2.0,0.619565,0.666667,1.098587,1.326667,1.773158,DRUG GM,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,0.271739,0.333333,1.0,0.0
1,313.0,318.0,16.0,1.0,0.510870,0.666667,1.063152,1.526667,2.081064,GROCERY,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,0.097826,NaN,0.0,0.0
2,406.0,31862.0,112.0,32.0,81.565217,23.333333,181.095109,113.100000,2.220249,PRODUCE,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,NaN,NaN,NaN,NaN
3,436.0,NaN,9.0,NaN,0.445652,NaN,0.701087,NaN,1.573171,DRUG GM,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,0.260870,NaN,0.0,0.0
4,367.0,406.0,113.0,105.0,298.097826,302.000000,286.356087,304.816667,0.960611,PRODUCE,National,16.75,19.666667,41.610109,44.786667,436,436.0,2.484186,0.423913,0.666667,1.0,1.0


In [ ]:
val_preds

array([9.91879571e-01, 2.01443294e-01, 2.90538696e-05, ...,
       2.34671906e-05, 2.39534378e-05, 5.55166806e-02])

In [ ]:
k = 5
targets_val_lvl_2_pred = targets_val_lvl_2.copy(deep=True)
targets_val_lvl_2_pred['predict_proba'] = val_preds
targets_val_lvl_2_pred = targets_val_lvl_2_pred[['user_id', 'item_id', 'predict_proba']]
users = targets_val_lvl_2_pred['user_id'].unique()
val_lvl_2_recommendations = pd.DataFrame(data = {'user_id': users})
val_lvl_2_recommendations['candidates'] = 0

def get_lvl_2_reccomendations(user_id, targets_val_lvl_2_pred):
  tdf = targets_val_lvl_2_pred[targets_val_lvl_2_pred['user_id']==user_id]
  tdf = tdf.drop(columns=['user_id'])
  tdf = tdf.groupby(['item_id']).max()
  tdf = tdf.sort_values(by='predict_proba', ascending=False)
  tdf = tdf.index[:k].tolist()
  return tdf

print(get_lvl_2_reccomendations(2027, targets_val_lvl_2_pred))
val_lvl_2_recommendations['candidates'] = val_lvl_2_recommendations['user_id'].apply(lambda x: get_lvl_2_reccomendations(x, targets_val_lvl_2_pred))

[1068719, 961554, 1055646, 923746, 981760]


In [ ]:
us_actual = set(actual_data_val_lvl_2.user_id.unique())
len(us_actual)

2042

In [ ]:
us_rec = set(val_lvl_2_recommendations.user_id.unique())
len(us_rec)

2042

In [ ]:
us_actual == us_rec

True

In [ ]:
mean, std = total_precision_at_k_pre(actual_data = actual_data_val_lvl_2, k=5, recommendations_table=val_lvl_2_recommendations)
print(mean, ' +- ', std)

0.3633512983831455  +-  0.29642544368594487


# Получили неплохое качество (смущает только разброс). Посмотрим, что это дает на лидерборде

In [ ]:
actual_data_lb = data_lb.groupby('user_id')['item_id'].unique().reset_index()
actual_data_lb.columns=['user_id', 'actual']
actual_data_lb.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [ ]:
df_val_lb = df_lb.copy(deep=True)  # Данные из модели первого уровня (обученные на всем датасете train)

Вариант - без переобучения модели второго уровня на всех данных датасета train

In [ ]:
df_val_lb.head()

,user_id,item_id
0,1,1049998
1,1,900875
2,1,1127831
3,1,852662
4,1,1082185


In [ ]:
data_lb_u.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [ ]:
to_X_val_lb = df_val_lb.copy(deep = True)  # Берем все пары user-item из модели первого уровня
print(to_X_val_lb.shape)

items_feats_train_lb_ = items_feats_train_lb[items_feats_train_lb.item_id.isin(to_X_val_lb.item_id.values)]
to_X_val_lb = to_X_val_lb.merge(items_feats_train_lb_, on='item_id', how='left')  # Добавляем items_features, натренированные на всем train
print(to_X_val_lb.shape)

users_feats_train_lb_ = users_feats_train_lb[users_feats_train_lb.user_id.isin(to_X_val_lb.user_id.values)]
to_X_val_lb = to_X_val_lb.merge(users_feats_train_lb_, on='user_id', how='left')  # Добавляем users_features, натренированные на всем train
print(to_X_val_lb.shape)

(188500, 2)
(188500, 13)
(190200, 20)


Прибавились записи по отсутствующему на train юзеру

In [ ]:
to_X_val_lb = merge_us_it(data_lb_u.copy(deep=True), hidden_weeks = 3, merge_to_data= to_X_val_lb.copy(deep=True))  # Хотя бля удобства работаем с объединенной таблицей, реально  тест (3 недели) скрыт
print(to_X_val_lb.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

(190200, 24)


In [ ]:
targets_val_lb.head()

,user_id,item_id,target
0,1,880007,1
1,1,883616,1
2,1,931136,1
3,1,938004,1
4,1,940947,1


In [ ]:
to_X_val_lb.head()

,user_id,item_id,frq_itm_store_id,3rw_frq_itm_store_id,qnt_itm_store_id,3rw_qnt_itm_store_id,wly_quantity,3rw_wly_quantity,itm_total_sales_value,3rw_itm_total_sales_value,itm_price,department,brand,basket_quantity,3rw_basket_quantity,total_sales_value,3rw_sales_value,frq_us_store_id,3rw_frq_us_store_id,avg_basket_sum,ui_total_quantity,3rw_ui_quantity,in_3_weeks,in_1_weeks
0,1,1049998,436.0,436.0,21.0,2.0,0.631579,1.000000,1.126737,1.990000,1.784000,DRUG GM,National,16.726316,16.0,41.683263,43.926667,436.0,436.0,2.492077,0.284211,0.666667,1.0,1.0
1,1,900875,313.0,NaN,16.0,NaN,0.494737,NaN,1.029579,NaN,2.081064,GROCERY,National,16.726316,16.0,41.683263,43.926667,436.0,436.0,2.492077,0.094737,NaN,0.0,0.0
2,1,1127831,406.0,31862.0,112.0,14.0,79.189474,6.333333,176.237895,27.283333,2.225522,PRODUCE,National,16.726316,16.0,41.683263,43.926667,436.0,436.0,2.492077,NaN,NaN,NaN,NaN
3,1,852662,436.0,NaN,9.0,NaN,0.431579,NaN,0.678947,NaN,1.573171,DRUG GM,National,16.726316,16.0,41.683263,43.926667,436.0,436.0,2.492077,0.252632,NaN,0.0,0.0
4,1,1082185,367.0,367.0,113.0,107.0,298.778947,319.666667,287.273895,315.420000,0.961493,PRODUCE,National,16.726316,16.0,41.683263,43.926667,436.0,436.0,2.492077,0.421053,0.333333,1.0,1.0


In [ ]:
X_val_lb = to_X_val_lb.drop(columns=['user_id', 'item_id'])
X_val_lb[cat_feats] = X_val_lb[cat_feats].astype('category')

val_preds_lb = lgb.predict_proba(X_val_lb)[:, 1]

k = 5
data_lb_pred = to_X_val_lb.copy(deep=True)
data_lb_pred['predict_proba'] = val_preds_lb
data_lb_pred = data_lb_pred[['user_id', 'item_id', 'predict_proba']]
users = data_lb_pred['user_id'].unique()
val_lvl_2_recommendations_lb = pd.DataFrame(data = {'user_id': users})
val_lvl_2_recommendations_lb['candidates'] = 0

print(get_lvl_2_reccomendations(2325, data_lb_pred))
val_lvl_2_recommendations_lb['candidates'] = val_lvl_2_recommendations_lb['user_id'].apply(lambda x: get_lvl_2_reccomendations(x, data_lb_pred))

[987562, 8090521, 5569230, 1081177, 1070015]


In [ ]:
mean, std = total_precision_at_k_pre(actual_data = actual_data_lb, k=5, recommendations_table=val_lvl_2_recommendations_lb)
print(mean, ' +- ', std)

0.32144373673036103  +-  0.2727286755848185


# Результат 32% выше проходного порога. Переучивать модель не будем.

Весь проект стандартное отклонение было очень большим. Так как задача не включала исследование этого вопроса, оставим пока это за скобками. Но настораживает. 

In [ ]:
val_lvl_2_recommendations_lb.shape

(1885, 2)

In [ ]:
data_lb.user_id.nunique()

1885

In [ ]:
val_lvl_2_recommendations_lb.head(10)

,user_id,candidates
0,1,"[995242, 961554, 9297615, 1074612, 1049998]"
1,2,"[916122, 1106523, 5569230, 1075368, 1133018]"
2,3,"[921345, 1092026, 1053690, 1022003, 1101173]"
3,6,"[995242, 948650, 840361, 845208, 1037863]"
4,7,"[961554, 860776, 1082185, 1106523, 13112721]"
5,8,"[1004906, 893501, 859075, 1005186, 1082185]"
6,9,"[995242, 1029743, 1106523, 1082185, 1098066]"
7,10,"[1106523, 1113588, 945779, 1012873, 1005186]"
8,13,"[1029743, 1082185, 826249, 857736, 951590]"
9,14,"[981760, 925205, 902377, 1029743, 1025611]"


In [ ]:
val_lvl_2_recommendations_lb.to_csv(OBJECTS_PATH + 'test_recommendations.csv')

In [ ]:
save_obj(val_lvl_2_recommendations_lb, 'test_recommendations')

'gdrive/My Drive/AI/GB/RecSys/capstone/objects/test_recommendations.pkl'